In [1]:
# ! pip install kgforge

In [2]:
from kgforge.core import KnowledgeGraphForge

In [3]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

# Imports

In [4]:
import json

In [5]:
from kgforge.core import Resource

In [6]:
def pp(x): print(json.dumps(x, indent=4))

## Resource to JSON

In [7]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [8]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [9]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [10]:
persons = [jane, john]

In [11]:
forge.register(persons)

<count> 2
<action> _register_one
<succeeded> True


In [12]:
association = Resource(type="Association", agent=persons)

In [13]:
forge.register(association)

<action> _register_one
<succeeded> True


In [14]:
pp(forge.as_json(association))

{
    "id": "1f58232d-2018-4037-98c2-b5310c32de52",
    "type": "Association",
    "agent": [
        {
            "id": "01b74d2e-a82d-49c0-ba16-818ef1ce51eb",
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva"
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "id": "26d226bb-9c29-4061-a1a3-ce659d0389de",
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}


In [15]:
pp(forge.as_json(jane, expanded=True))

{
    "@id": "file:///Users/agarcia/Developments/kgforge/examples/notebooks/getting-started/01b74d2e-a82d-49c0-ba16-818ef1ce51eb",
    "@type": "schema:Person",
    "schema:name": "Jane Doe"
}


In [16]:
pp(forge.as_json(jane, store_metadata=True))

{
    "id": "01b74d2e-a82d-49c0-ba16-818ef1ce51eb",
    "type": "Person",
    "address": {
        "type": "PostalAddress",
        "country": "Switzerland",
        "locality": "Geneva"
    },
    "email": "(missing)",
    "name": "Jane Doe",
    "deprecated": false,
    "version": 1
}


## JSON to Resource

In [17]:
address = Resource(type="PostalAddress", country="Switzerland", locality="Geneva")

In [18]:
jane = Resource(type="Person", name="Jane Doe", address=address, email="(missing)")

In [19]:
john = Resource(type="Person", name="John Smith", email="john.smith@epfl.ch")

In [20]:
persons = [jane, john]

In [21]:
association = Resource(type="Association", agent=[jane, john])

In [22]:
data = {
    "type": "Association",
    "agent": [
        {
            "type": "Person",
            "address": {
                "type": "PostalAddress",
                "country": "Switzerland",
                "locality": "Geneva",
            },
            "email": "(missing)",
            "name": "Jane Doe"
        },
        {
            "type": "Person",
            "email": "john.smith@epfl.ch",
            "name": "John Smith"
        }
    ]
}

In [23]:
resource = forge.from_json(data)

In [24]:
resource == association

True

In [25]:
print(forge.from_json(data, na="(missing)"))

{
    type: Association
    agent:
    [
        {
            type: Person
            address:
            {
                type: PostalAddress
                country: Switzerland
                locality: Geneva
            }
            name: Jane Doe
        }
        {
            type: Person
            email: john.smith@epfl.ch
            name: John Smith
        }
    ]
}
